Things to do:
* Date/Time
    * verify dates are real (what to do with dates that are strange, e.g., pre-1900?)
* split date and time (?)
* convert times to UTC
* duration
    * split number and unit(e.g., seconds, minutes
    * convert duration to milliseconds
* location
    * pull all non-location information out of location fields (i.e., city, state)
    * add country column
    * validate all 3 columns
* summary information
    * creates columns for nouns, adjectives, verbs (?)
    * extra colors (new column)
* factorize
    * colors
    * shapes
   
References:

https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/

https://blog.dataiku.com/text-classification-the-first-step-toward-nlp-mastery

https://thinkinfi.com/complete-guide-for-natural-language-processing-in-python/

https://towardsdatascience.com/nlp-in-python-vectorizing-a2b4fc1a339e

In [1]:
import pandas as pd
import numpy as np
import time
import logging
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
!pip install contractions
import contractions

!pip install country_list
import country_list

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
logging.basicConfig(filename='sightings_cleaning.log', format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)
logger = logging.getLogger()

In [3]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split(' ')])

def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   

    return ' '.join(expanded_words)

def regex_clean(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

def clean_text(text):
    stop = set(nltk.corpus.stopwords.words('english'))
    cleaned = expand_contractions(text.lower())
    # cleaned = regex_clean(text)
    tokens = word_tokenize(cleaned)
    cleaned = ' '.join([w for w in tokens if not w in stop])
    cleaned = lemmatize_text(cleaned)
    return cleaned

In [4]:
def load_clean_sightings_dataframe():
    file_name = "sightings.pkl"
    logger.info(f"Data read from {file_name}")
    sightings = pd.read_pickle(file_name)

    sightings = sightings[sightings['Summary'].str.contains('MADAR')==False]
    
    # The Detail_Summary column needs to be cleaned for the cleaning function to work.
    sightings["Detail_Summary"] = sightings["Detail_Summary"].fillna("")
    sightings.loc[sightings["Detail_Summary"] == "Summary detail page not found.", "Detail_Summary"] = ""
    
    print(f"Sightings: {len(sightings)}")

    sightings_cleaned = sightings.copy()
    sightings_cleaned['Detail_Summary_nltk'] = sightings_cleaned['Detail_Summary'].apply(clean_text)
    
    print(f"Cleaned Sightings: {len(sightings_cleaned)}")
    sightings_cleaned.to_pickle(cleaned_file_name)

In [5]:
cleaned_file_name = "sightings_cleaned.pkl"
start_fresh = False

if start_fresh:
    load_clean_sightings_dataframe()

In the next block we save the data to a file. The initial cleaning is on +95k records. This takes some time. In the remaining cleaning, if one messes up, as I have done quite a few times, they can re-run pd.read_pickle() which will restore the dataframe to the "just processed" phase.

In [6]:
sightings_cleaned = pd.read_pickle(cleaned_file_name)

In [7]:
sightings_cleaned.head(15)

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk
0,4/23/21 06:30,Blackshear,GA,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting
1,4/23/21 06:00,Mechanicsville,VA,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...
2,4/23/21 06:00,Vero Beach,FL,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...
3,4/23/21 05:59,St. Augustine,FL,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...
4,4/23/21 05:58,Durham,NC,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...
5,4/23/21 05:55,I-16 south,GA,Sphere,10 minutes,Noticed a intense light that was covering a la...,4/23/21,http://www.nuforc.org/webreports/162/S162823.html,\nDriving on I-16 south and noticed a intense ...,driving i-16 south noticed intense light cover...
6,4/23/21 05:54,Parrish,FL,Light,5 minutes,Two bright lights one flashing with a descendi...,4/23/21,http://www.nuforc.org/webreports/162/S162820.html,\nTwo bright lights one flashing with a descen...,two bright light one flashing descending expan...
7,4/23/21 05:45,Champions Gate,FL,Light,~10-15 minutes,Im former military and have never seen aircraf...,4/23/21,http://www.nuforc.org/webreports/162/S162826.html,\nIm former military and have never seen aircr...,I former military never seen aircraft that.inc...
8,4/23/21 05:45,Belleview,FL,Diamond,15-20 minutes,((NUFORC Note: Rocket launch from Cape Canav...,4/23/21,http://www.nuforc.org/webreports/162/S162821.html,\n ((NUFORC Note: Rocket launch from Cape Can...,( ( nuforc note : rocket launch cape canaveral...
9,4/23/21 02:40,Firestone,CO,Chevron,3-4 seconds,"I witnessed a chevron-shaped object, silent an...",4/23/21,http://www.nuforc.org/webreports/162/S162827.html,"\nI witnessed a chevron-shaped object, silent ...","witnessed chevron-shaped object , silent seven..."


In [8]:
len(sightings_cleaned)

95690

Do we want to remove the ones below with NUFORC notes? Perhaps someone could go through them to see what the notes are. Things like Starlink sightings should be removed. For now, the following block removes them otherwise nuforc and note dominate the wordcloud.

In [9]:
sightings_cleaned = sightings_cleaned[sightings_cleaned['Detail_Summary_nltk'].str.contains('nuforc note')==False]
sightings_cleaned = sightings_cleaned[sightings_cleaned['Detail_Summary_nltk'].str.contains('NUFORC note')==False]
sightings_cleaned = sightings_cleaned[sightings_cleaned['Detail_Summary_nltk'].str.contains('nuforc')==False]
sightings_cleaned = sightings_cleaned[sightings_cleaned['Detail_Summary_nltk'].str.contains('NUFORC')==False]
len(sightings_cleaned)

67841

In [10]:
sightings_cleaned.head(10)

,Date_Time,City,State,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk
0,4/23/21 06:30,Blackshear,GA,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting
1,4/23/21 06:00,Mechanicsville,VA,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...
2,4/23/21 06:00,Vero Beach,FL,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...
3,4/23/21 05:59,St. Augustine,FL,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...
4,4/23/21 05:58,Durham,NC,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...
5,4/23/21 05:55,I-16 south,GA,Sphere,10 minutes,Noticed a intense light that was covering a la...,4/23/21,http://www.nuforc.org/webreports/162/S162823.html,\nDriving on I-16 south and noticed a intense ...,driving i-16 south noticed intense light cover...
6,4/23/21 05:54,Parrish,FL,Light,5 minutes,Two bright lights one flashing with a descendi...,4/23/21,http://www.nuforc.org/webreports/162/S162820.html,\nTwo bright lights one flashing with a descen...,two bright light one flashing descending expan...
7,4/23/21 05:45,Champions Gate,FL,Light,~10-15 minutes,Im former military and have never seen aircraf...,4/23/21,http://www.nuforc.org/webreports/162/S162826.html,\nIm former military and have never seen aircr...,I former military never seen aircraft that.inc...
9,4/23/21 02:40,Firestone,CO,Chevron,3-4 seconds,"I witnessed a chevron-shaped object, silent an...",4/23/21,http://www.nuforc.org/webreports/162/S162827.html,"\nI witnessed a chevron-shaped object, silent ...","witnessed chevron-shaped object , silent seven..."
10,4/22/21 22:23,New York City (Brooklyn),NY,Fireball,2 minutes,Saw a steady pulsating fireball above that mov...,4/23/21,http://www.nuforc.org/webreports/162/S162818.html,\nSaw a steady pulsating fireball above that m...,saw steady pulsating fireball moved slowly awa...


The next section is cleaning up the city, state, and country column. To begin, we take cities that have () in them. We split that. Some of the parentheses have country or state in them, however, many do not have anything useful. Once cleaned, that needs to be merged back in and the general task of cleaning up the remaining location data may go forward.

In [11]:
# Backup the City column in case the notes may be useful
sightings_cleaned["Notes"] = sightings_cleaned[sightings_cleaned['City'].str.contains('\(')].City

In [12]:
sightings_cleaned[sightings_cleaned['City'].str.contains('\(')].City

10                    New York City (Brooklyn)
11                           Firozabad (India)
24                    New York City (Brooklyn)
33                            Nanaimo (Canada)
95                     Merseyside (UK/England)
148                      Melbourne (Australia)
152                          Winnipeg (Canada)
203                         Gilching (Germany)
205                        Chilliwack (Canada)
215                          Langford (Canada)
223                     Vancouver Bc  (Canada)
306                           Oakbank (Canada)
309                         Rugby (UK/England)
344                      Tamworth (UK/England)
349                      New York City (Bronx)
365                           Calgary (Canada)
382                     Northwich (UK/England)
388                    Rosarito)(Baja)(Mexico)
398                         Monterrey (Mexico)
436                           Wiarton (Canada)
446                 Littlehampton (UK/England)
512          

In [13]:
# df is a temporary dataframe so that I can clean cities with a () in them. The following is all of that work.
# Eventually, this could be merged into the main dataframe, or this code could be applied to that dataframe
# when we are confident it works.
df = sightings_cleaned[sightings_cleaned['City'].str.contains('\(')].City.str.split("\(([^)]+)", expand= True)
try:
    df.columns = ["City", "Country", "EndParenth", "Empty1", "Empty2", "Empty3", "Empty4"]
except:
    df.columns = ["City", "Country", "EndParenth", "Empty1", "Empty2"]
df.drop(["EndParenth"], axis=1, inplace = True)
df["City"] = df["City"].str.strip()
df

for index, row in df.iterrows():
    df.loc[index, "State"] = sightings_cleaned.loc[index].State

In [14]:
df["ignore"] = False

In [15]:
df

,City,Country,Empty1,Empty2,Empty3,Empty4,State,ignore
10,New York City,Brooklyn,None,None,None,None,NY,False
11,Firozabad,India,None,None,None,None,,False
24,New York City,Brooklyn,None,None,None,None,NY,False
33,Nanaimo,Canada,None,None,None,None,BC,False
95,Merseyside,UK/England,None,None,None,None,,False
148,Melbourne,Australia,None,None,None,None,,False
152,Winnipeg,Canada,None,None,None,None,MB,False
203,Gilching,Germany,None,None,None,None,,False
205,Chilliwack,Canada,None,None,None,None,BC,False
215,Langford,Canada,None,None,None,None,BC,False


In [16]:
countries = dict(country_list.countries_for_language('en'))
cities_file = "us_cities_states_counties.csv"
cities_df = pd.read_csv(cities_file, delimiter="|")
city_list = cities_df.City.unique().tolist()

cities_df["State short"].unique()

array(['NY', 'PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'AE',
       'PA', 'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', nan,
       'AL', 'TN', 'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD',
       'ND', 'MT', 'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO',
       'WY', 'ID', 'UT', 'AZ', 'NM', 'NV', 'CA', 'AP', 'HI', 'AS', 'GU',
       'PW', 'FM', 'MP', 'MH', 'OR', 'WA', 'AK'], dtype=object)

In [17]:
# This record is "NA" for the country. Have to fix that or the next few things will throw an error.
# Ask me how I know that.
df.loc[89338, "Country"] = "USA"

In [18]:
# Do we want these as England, Wales, etc.?
df.loc[df['Country'].str.contains('UK'), "Country"] = "United Kingdom"
df.loc[df['Country'].str.contains('Northern Ireland'), "Country"] = "United Kingdom"
df.loc[df['City'].str.contains('UK/England'), "Country"] = "United Kingdom"
df.loc[df['City'].str.contains('UK/England'), "City"] = ""
df.loc[df['Country'].str.contains('UK'), 'Country'].unique()

array([], dtype=object)

In [19]:
df.loc[(df['Country'].isin(countries.values())==False)&(df.Empty1.isnull()==False), "Country"] = df["Empty1"]
df.loc[df.Empty1 == "Canada", "Country"] = "Canada"

In [20]:
df.loc[df['Country'].str.contains('Brooklyn'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Bronx'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Brookline'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Westchester County'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Baja'), "Country"] = "Mexico"
df.loc[df['Country'].str.contains('Manhattan'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Bronx'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Queens'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Watts'), "Country"] = "United States"
df.loc[df['City'].str.contains('Warsaw/Clinton'), "Country"] = "United States"
df.loc[df['Country'].str.contains('USA'), "Country"] = "United States"
df.loc[df['Country'].str.contains('Calgary'), "Country"] = "Canada"
df.loc[df['Country'].str.contains('Wilhelmsburg'), "Country"] = "Germany"
df.loc[df['Country'].str.contains('German'), "Country"] = "Germany"
df.loc[df['Country'].str.contains('Czech Republic'), "Country"] = "Czechia"
df.loc[df['Country'].str.contains('Punjab'), "Country"] = "India"
df.loc[df['Country'].str.contains('West Germany'), "Country"] = "Germany"
df.loc[df['Country'].str.contains('Brasil'), "Country"] = "Brazil"
df.loc[df['Country'].str.contains('Macedonia'), "Country"] = "North Macedonia"
df.loc[df['Country'].str.contains('México'), "Country"] = "Mexico"
df.loc[df['Country'].str.contains('Western Australia'), "Country"] = "Australia"
df.loc[97083, "City"] = "Boston"
df.loc[df['Country'].str.contains('Bosnia'), 'Country'] = "Bosnia & Herzegovina"
df.loc[(df['Country'].str.contains("Australia")), "Country"] = "Australia"
df.loc[(df['Country'].str.contains("Australi")), "Country"] = "Australia"
df.loc[(df['Country'].str.contains("Rep. of Ireland")), "Country"] = "Ireland"
df.loc[(df['Country'].str.contains("Republic of Ireland")), "Country"] = "Ireland"
df.loc[df['Country'].str.contains('U.A.E.'), "Country"] = "United Arab Emirates"
df.loc[df['Country'].str.contains('Oman/UAE'), "Country"] = "United Arab Emirates"
df.loc[df['Country'].str.contains('UAE'), "Country"] = "United Arab Emirates"
df.loc[df['Country'].str.contains('UAR'), "Country"] = "United Arab Emirates"
df.loc[df['Country'].str.contains('U.A.R.'), "Country"] = "United Arab Emirates"
df.loc[df['Country'].str.contains('Dubai'), "Country"] = "United Arab Emirates"
df.loc[(df['City']=="Leduc"), "Country"] = "Canada"
df.loc[df['Country'].str.contains('Surinam'), "Country"] = "Suriname"
df.loc[df['Country'].str.contains('St. Helena Island'), "Country"] = "St. Helena"
df.loc[df['Country'].str.contains('El Poblado'), "City"] = "Medellin"
df.loc[df['Country'].str.contains('El Poblado'), "Country"] = "Colombia"
df.loc[df['Country'].str.contains('Tobago'), "Country"] = "Trinidad & Tobago"
df.loc[df['Country'].str.contains('Trinidad'), "Country"] = "Trinidad & Tobago"
df.loc[df['Country'].str.contains('Viet nam'), "Country"] = "Vietnam"
df.loc[df['Country'].str.contains('Viet Nam'), "Country"] = "Vietnam"
df.loc[df['Country'].str.contains('Hidalgo'), "State"] = "Hidalgo"
df.loc[df['Country'].str.contains('Hidalgo'), "Country"] = "Mexico"
df.loc[df['Country'].str.contains('Netherlands'), "Country"] = "Netherlands"
df.loc[df['City'].str.contains("U. S."), "Country"] = "United States"
df.loc[(df['City'].str.contains("U. S.")), "State"] = ""
df.loc[(df['City'].str.contains("U. S.")), "City"] = ""
df.loc[(df['Country'].str.contains("Nauru")), "Country"] = "Nauru"
df.loc[(df['Country'].str.contains("Menorca")), "State"] = "Menorca"
df.loc[(df['Country'].str.contains("Menorca")), "Country"] = "Spain"
df.loc[(df['Country'].str.contains("Riu Palace Hotel")), "City"] = "Playa Matapalo"
df.loc[(df['Country'].str.contains("Riu Palace Hotel")), "Country"] = "Costa Rica"
df.loc[(df['City'].str.contains("Australia")), "Country"] = "Australia"
df.loc[(df['City'].str.contains("Australia")), "City"] = ""
df.loc[(df['Country'].str.contains("Cyprus")), "Country"] = "Cyprus"
df.loc[(df['Country'].str.contains("Oman")), "Country"] = "Oman"
df.loc[(df['Country'].str.contains("Gibralter")), "Country"] = "Gibralter"
df.loc[(df['City'].str.contains("Farallon")), "Country"] = "Panama"
df.loc[(df['Country'].str.contains("Guatamala")), "Country"] = "Guatemala"
df.loc[(df['Country'].str.contains("Cayman")), "Country"] = "Cayman Islands"
df.loc[(df['City'].str.contains("Cayman")), "Country"] = "Cayman Islands"
df.loc[(df['City'].str.contains("Cayman")), "City"] = ""
df.loc[(df['Country'].str.contains("Virgin Islands")), "Country"] = "U.S. Virgin Islands"
df.loc[(df['City'].str.contains("Maldives")), "Country"] = "Maldives"
df.loc[(df['City'].str.contains("Maldives")), "City"] = "Meerufenfushi island"
df.loc[(df['Country'].str.contains("Papua/New Guinea")), "Country"] = "Papua New Guinea"
df.loc[(df['Country'].str.contains("Yorkshire")), "State"] = "Yorkshire"
df.loc[(df['Country'].str.contains("Yorkshire")), "City"] = ""
df.loc[(df['Country'].str.contains("Yorkshire")), "Country"] = "United Kingdom"
df.loc[(df['Country'].str.contains("Kazakstan")), "Country"] = "Kazakhstan"
df.loc[(df['Country'].str.contains("Taiwan")), "Country"] = "Taiwan"
df.loc[(df['Country'].str.contains("Hampshire")), "State"] = "Hampshire"
df.loc[(df['Country'].str.contains("Hampshire")), "Country"] = "United Kingdom"
df.loc[(df['City'].str.contains("Atlantic Ocean")), "Country"] = "Atlantic Ocean"
df.loc[(df['City'].str.contains("Atlantic Ocean")), "City"] = ""
df.loc[(df['City'].str.contains("Pacific Ocean")), "Country"] = "Pacific Ocean"
df.loc[(df['City'].str.contains("Pacific Ocean")), "City"] = ""
df.loc[(df['City'].str.contains("Indian Ocean")), "Country"] = "Indian Ocean"
df.loc[(df['City'].str.contains("Indian Ocean")), "City"] = ""
df.loc[(df['City'].str.contains("SE Arizona")), "State"] = "AZ"
df.loc[(df['City'].str.contains("SE Arizona")), "Country"] = "United States"
df.loc[(df['City'].str.contains("SE Arizona")), "City"] = ""
df.loc[(df['City'].str.contains("Dominican Republic")), "Country"] = "Dominican Republic"
df.loc[(df['City'].str.contains("Dominican Republic")), "City"] = ""
df.loc[(df['City'].str.contains("Rockhampton")), "Country"] = "Australia"
df.loc[(df['Country'].str.contains('Camp "New Jersey"')), "Country"] = "Kuwait"
df.loc[(df['Country'].str.contains('Camp "New Jersey"')), "City"] = ""
df.loc[(df['City'].str.contains("Shanghai")), "City"] = "Shanghai"
df.loc[(df['City'].str.contains("Shanghai")), "Country"] = "China"
df.loc[(df['City'].str.contains("Bloemfontein")), "Country"] = "South Africa"
df.loc[(df['City'].str.contains("France")), "Country"] = "France"
df.loc[(df['Country'].str.contains("North part of France")), "Country"] = "France"
df.loc[(df['City'].str.contains("France")), "City"] = ""
df.loc[(df['City'].str.contains("Urmston")), "State"] = "Manchester"
df.loc[(df['City'].str.contains("Urmston")), "Country"] = "United Kingdom"
df.loc[(df['City'].str.contains("Sweden")), "Country"] = "Sweden"
df.loc[(df['City'].str.contains("Sweden")), "City"] = ""
df.loc[(df['City'].str.contains("Songtan/Pyongtaek")), "Country"] = "South Korea"
df.loc[(df['City'].str.contains("Pyongtaek")), "City"] = "Pyongtaek"
df.loc[(df['City'].str.contains("Seoul")), "Country"] = "South Korea"
df.loc[(df['City'].str.contains("Afghanistan")), "Country"] = "Afghanistan"
df.loc[(df['City'].str.contains("Afghanistan")), "City"] = ""
df.loc[(df['Country'].str.contains("Curacao")), "City"] = "Curacao"
df.loc[(df['Country'].str.contains("Curacao")), "Country"] = "Caribbean Netherlands"
df.loc[(df['City'].str.contains("Praia")), "Country"] = "Cape Verde"
df.loc[(df['City'].str.contains("Faliraki")), "Country"] = "Greece"
df.loc[(df['City'].str.contains("Argentina")), "Country"] = "Argentina"
df.loc[(df['City'].str.contains("Argentina")), "City"] = ""
df.loc[(df['Country'].str.contains("Ochorios")), "City"] = "Ochorios"
df.loc[(df['Country'].str.contains("Ochorios")), "Country"] = "Jamaica"
df.loc[(df['Country'].str.contains("Myanmar")), "Country"] = "Myanmar (Burma)"
df.loc[(df['Country'].str.contains("Phoenix to NYC")), "Country"] = "United States"
df.loc[(df['Country'].str.contains("Turks & Caicos")), "Country"] = "Turks & Caicos Islands"
df.loc[(df['Country'].str.contains("Italy/Greece")), "Country"] = "Italy"
df.loc[(df['City'].str.contains("Hermosillo/Obregon City")), "Country"] = "Mexico"
df.loc[(df['City'].str.contains("Hermosillo/Obregon City")), "City"] = "Ciudad Obregón"
df.loc[(df['City'].str.contains("Manchester area")), "Country"] = "United Kingdom"
df.loc[(df['City'].str.contains("Manchester area")), "City"] = "Manchester"
df.loc[(df['Country'].str.contains("U. S. and Canada")), "Country"] = "United States, Canada"
df.loc[(df['City'].str.contains("Vancouver Bc")), "State"] = "British Columbia"
df.loc[(df['City'].str.contains("Vancouver Bc")), "City"] = "Vancouver"
df.loc[(df['City'].str.contains("Barksdale AFB")), "City"] = "Bossier City"


In [21]:
df[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City))].Empty1.unique()

array(['in-flight', None, '"Abiquiu"', 'Northern Ireland', 'UK/England',
       'UK/Scotland', 'Victoria', 'pilot report', 'Riverside',
       'Republic of South Africa', 'UK/Wales', 'near', 'in flight',
       'Boston'], dtype=object)

In [22]:
df[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City))].Empty2.unique()

array([')', None, ''], dtype=object)

In [23]:
df[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City))].Empty3.unique()

array([None, 'Australia'], dtype=object)

In [24]:
df[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City))].Empty4.unique()

array([None, ')'], dtype=object)

In [25]:
# Country not in Country list
# Cities not in the US city list
# cities_df[cities_df.City == "Canada"] ----> Canada is a city in Kentucky
# df[(df['Country'].isin(countries.values())==False)]

# Cities in the US city list
#df[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City)), "Country"] = "United States"
df.loc[df['Empty1'].isnull(), "Empty1"] = " "
df.loc[df['Empty2'].isnull(), "Empty2"] = " "
df.loc[df['Empty3'].isnull(), "Empty3"] = " "
df.loc[df['Empty4'].isnull(), "Empty4"] = " "
df.loc[df['Empty3'].str.contains('Australia'), "Country"] = "Australia"
df.loc[df['Empty1'].str.contains('Northern Ireland'), "Country"] = "United Kingdom"
df.loc[df['Empty1'].str.contains('UK/England'), "Country"] = "United Kingdom"
df.loc[df['Empty1'].str.contains('UK/Wales'), "Country"] = "United Kingdom"
df.loc[df['Empty1'].str.contains('UK/Scotland'), "Country"] = "United Kingdom"
df.loc[df['Empty1'].str.contains('Republic of South Africa'), "Country"] = "South Africa"


In [26]:
df.drop(["Empty1", "Empty2", "Empty3", "Empty4"], axis=1, inplace=True)

In [27]:
df.loc[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City)), "Country"] = "United States"
df.loc[(df['Country'].isin(countries.values())==False)&(df['State'].isin(cities_df["State short"])==True)]

# Country not in the list of countries
# State is the list of US cities State field
# Changing the Country to United States
df.loc[(df['Country'].isin(countries.values())==False)&(df['State'].isin(cities_df["State short"])==True), "Country"] = "United States"

In [28]:
df.loc[(df['Country'].isin(countries.values())==False)&(df['City'].isin(cities_df.City)==False)&df.Country.str.contains(";")]

df.loc[46878, "Country"] = "Lesotho"
df.loc[23766, "Country"] = "Japan"
df.loc[23766, "City"] = "Hiroshima"
df.loc[27598, "Country"] = "India"
df.loc[58864, "Country"] = "Russia"
df.loc[71552, "Country"] = "Tunisia"
df.loc[71552, "City"] = ""
df.loc[84105, "Country"] = "Thailand"
df.loc[96267, "Country"] = "Vietnam"
df.loc[96267, "City"] = "Vietnam"
df.loc[94772, "Country"] = "Australia"
df.loc[38099, "Country"] = "Israel"
df.loc[85560, "Country"] = "U.S. Virgin Islands"
df.loc[7900, "Country"] = "U.S. Virgin Islands"
df.loc[94486, "City"] = ""
df.loc[94486, "Country"] = "France"
df.loc[79771, "City"] = ""
df.loc[79771, "Country"] = ""
df.loc[92408, "Country"] = "Laos"
df.loc[92408, "City"] = "Luang Phabang"
df.loc[7219, "Country"] = "Costa Rica"
df.loc[12183, "Country"] = "Canada"
df.loc[12183, "State"] = "British Columbia"
df.loc[7075, "Country"] = "Canada"
df.loc[7075, "State"] = "Quebec"
df.loc[848, "Country"] = "United Arab Emirates"
df.loc[7924, "Country"] = "Panama"
df.loc[8131, "Country"] = "Canada"
df.loc[80866, "Country"] = ""
df.loc[80866, "City"] = ""
df.loc[80866, "State"] = ""


In [29]:
df.loc[(df['City'].str.contains("Ocean"))&(df['City'].str.contains("Ocean City")==False), "State"] = ""
df.loc[(df['City'].str.contains("Ocean"))&(df['City'].str.contains("Ocean City")==False), "Country"] = ""
df.loc[(df['City'].str.contains("Ocean"))&(df['City'].str.contains("Ocean City")==False)]

,City,Country,State,ignore
30818,Oceanside,,,False
40870,Oceanside,,,False


In [30]:
countries

{'AF': 'Afghanistan',
 'AX': 'Åland Islands',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AD': 'Andorra',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AQ': 'Antarctica',
 'AG': 'Antigua & Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia',
 'BA': 'Bosnia & Herzegovina',
 'BW': 'Botswana',
 'BV': 'Bouvet Island',
 'BR': 'Brazil',
 'IO': 'British Indian Ocean Territory',
 'VG': 'British Virgin Islands',
 'BN': 'Brunei',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'CV': 'Cape Verde',
 'BQ': 'Caribbean Netherlands',
 'KY': 'Cayman Islands',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'CN': 'China',
 'CX': 'Christmas Isla

In [31]:
#df.loc[(df['City'].str.contains("Shanghai"))]

sightings_cleaned.loc[24755]

# df.loc[(df['Country'].str.contains("|".join(keywords)))]

Date_Time                                                  9/13/16 03:00
City                                             Adriatic (Italy/Greece)
State                                                                   
Shape                                                             Circle
Duration                                                      20 minutes
Summary                .3 X Cream/white solid circle shape objects mo...
Posted                                                           9/30/16
Detail_Link            http://www.nuforc.org/webreports/130/S130197.html
Detail_Summary         \n.3 X Cream/white solid circle shape objects ...
Detail_Summary_nltk    .3 x cream/white solid circle shape object mov...
Notes                                            Adriatic (Italy/Greece)
Name: 24755, dtype: object

In [32]:
keywords = ["in flight", "inflight", "at sea", "between", "unknown", "oil rig", "ship", "unspecified", "ISS", "in most"]
df.loc[(df['Country'].isin(countries.values())==False)&(df['Country'].str.contains("|".join(keywords))), "ignore"] = True

In [33]:
df.loc[(df['Country'].isin(countries.values())==False), "ignore"]= True

In [34]:
for index, row in df[df['City'].isin(cities_df["City alias"])].iterrows():
    df.loc[index, "city alias"] = df.loc[index, "City"]
    df.loc[index, "City"] = cities_df.loc[cities_df['City alias'] == row.City, "City"].tolist()[0]

In [35]:
df

,City,Country,State,ignore,city alias
10,New York,United States,NY,False,New York City
11,Firozabad,India,,False,NaN
24,New York,United States,NY,False,New York City
33,Nanaimo,Canada,BC,False,NaN
95,Merseyside,United Kingdom,,False,NaN
148,Melbourne,Australia,,False,Melbourne
152,Winnipeg,Canada,MB,False,NaN
203,Gilching,Germany,,False,NaN
205,Chilliwack,Canada,BC,False,NaN
215,Langford,Canada,BC,False,Langford


In [36]:
df_merge = df[df.ignore == False]

In [37]:
sightings_cleaned_copy = sightings_cleaned.copy()
sightings_cleaned_copy = sightings_cleaned_copy.join(df_merge[["City", "State", "Country"]], lsuffix='_left', rsuffix='_right')
sightings_cleaned_copy.loc[sightings_cleaned_copy["City_right"].isnull() == False, "City_left"] = sightings_cleaned_copy.loc[sightings_cleaned_copy["City_right"].isnull() == False, "City_right"]
sightings_cleaned_copy.loc[sightings_cleaned_copy["State_right"].isnull() == False, "State_left"] = sightings_cleaned_copy.loc[sightings_cleaned_copy["State_right"].isnull() == False, "State_right"]
sightings_cleaned_copy["City"] = sightings_cleaned_copy["City_left"]
sightings_cleaned_copy["State"] = sightings_cleaned_copy["State_left"]
sightings_cleaned_copy.drop(["City_right", "City_left", "State_right", "State_left"], axis=1, inplace=True)

# Delete ignored records
# Records not processed need to be cleaned...
# add country/state where not present

In [38]:
sightings_cleaned = sightings_cleaned_copy.copy()

In [39]:
len(sightings_cleaned)

67841

df needs to be merged back into sightings_cleaned then saved to the file.

In [40]:
sightings_cleaned.to_pickle(cleaned_file_name)

In [41]:
sightings_cleaned.head()

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State
0,4/23/21 06:30,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting,NaN,NaN,Blackshear,GA
1,4/23/21 06:00,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...,NaN,NaN,Mechanicsville,VA
2,4/23/21 06:00,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...,NaN,NaN,Vero Beach,FL
3,4/23/21 05:59,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...,NaN,NaN,St. Augustine,FL
4,4/23/21 05:58,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...,NaN,NaN,Durham,NC
